## Paired Switching

The idea behind this strategy is that if the assets are negatively correlated, then a traditional mixed portfolio might lead to a lower return than the return for the individual assets.

We periodically compute the Co-relation Coefficients' Matrix and choose pair of stocks which have the most negative co-relations and trade in them. The basic idea being that if one of them would fall, then the other would rise and hence, we must switch between them!

We need a re-balancing period ,i.e. The time after which we would be again calculating the co-relations and taking a different pair of stocks : **T**

We would also need to keep switching between these selected stocks in a pair at a certain interval of time : **P**

Next, We need to specify how many such pairs would we be trading in : **M**

The Fraction of the total balance we would be investing on every turn : **R**

The Brokerage Fee as a fraction of the total transaction amount : **F**



In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

T = 60
P = 7
M = 4
R = 0.8
F = 0.005

### Few Upcoming Functions(GetData and PartitionData) would be similar to the ones you encountered in the Momentum Assignment (Feel free to Copy-Paste your code from there!)

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!) 

In [ ]:
def GetData(NameOfFile):
  return pd.DataFrame(pd.read_csv(NameOfFile, usecols = ['datadate','tic','adjcp']))

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later. 

In [ ]:
def PartitionData(Data):
  DateToIndex = {}
  for e in range(len(Data['datadate'])):
    DateToIndex.setdefault(str(Data['datadate'][e]),e)
  return [np.array_split(Data,2926), DateToIndex]

def GetMomentumBasedPriority(PartitionedDataFrameList, DateToIndex ,today):
  NdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -N)
  i = 0
  while i >= 0:
    x = str(NdaysAgo - datetime.timedelta(days = i)).replace('-','')
    if (x in DateToIndex.keys()):
      break
    i += 1
  temp = x
  Momentum = np.array(PartitionedDataFrameList[DateToIndex[today] % 2926]['adjcp']) - np.array(PartitionedDataFrameList[DateToIndex[temp] % 2926]['adjcp'])
  Sum = np.array(PartitionedDataFrameList[DateToIndex[today] % 2926]['adjcp'])
  for i in range((DateToIndex[temp] + 1), (DateToIndex[today]),30):
    Sum += np.array(PartitionedDataFrameList[i % 2926]['adjcp'])
  return Momentum * N / Sum

def GetBalanced(prices, weights, balance):
  copy = np.flip(np.sort(weights))
  for i in range(M,len(weights)):
    copy[i] = 0
  for i in range(len(weights)):
    if weights[i] not in copy:
      weights[i] = 0
    elif weights[i] < 0:
      weights[i] = 0
  sum = np.sum(weights)
  if (sum <= 0):
    return np.zeros(30, dtype = float)
  weights /= sum
  sum = np.sum(weights * prices)
  return (balance / sum) * weights

Now, We need a function which takes as input a pair of stocks (as indices in range 0-29), Today's date,  alongwith the Partitioned DataFrames which will tell us which of the two stocks has a better outlook for the future, and we'll simply move all our Money from one of them to the other. 

This will be done once in every **P** trading sessions.

You could choose between the two stocks in a number of ways, the simplest of which can be what you did in the last assignment, the Momentum Indicator (i.e. Take the difference between the prices of the stocks in a certain number of days and do your deed! Remember this number of days **N** must be <= **T**)

You may also use other indicators like [RSI](https://www.investopedia.com/terms/r/rsi.asp), [Stochs](https://www.investopedia.com/terms/s/stochasticoscillator.asp), [MACD](https://www.investopedia.com/terms/m/macd.asp) or anything else! (If you decide to use MACD or RSI, you could either implement other functions to calculate them or simply change your ParitionData() function to get the MACD and RSI values as they are already stored there for you!)

You need not worry about getting the nearest dates (or dates at all) in this case, because all you care about is co-relations in long term and growth in short-term so it doesn't matter if you're ahead or behind a few days!

'today' would be an index(integer), i.e. an index on the PartitionedDataFrames list denoting where we are at currently!

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!


In [ ]:
def Switch(firstStock, secondStock, today ,PartitionedDataFrames):
  weights = GetMomentumBasedPriority(PartitionedDataFrames, DateToIndex, today)
  if weights[firstStock] >= weights[secondStock]:
    return firstStock
  else:
    return secondStock

Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

(Again some of the functions here, would be same as the previous assignment, so feel free to Copy-Paste)

Features : 


1.   Your Initial Balance
2.   Your Current Balance (Money which isn't invested!)
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)
5.   A list of **M** or less(as it might be that less than **M** stocks are negatively co-related) Tuples of indices (i,j) where i and j are the indices (from 0-29) of stocks which are negatively co-related.

Functionalities: 



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!) 
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Change Pairs (**T** period intervals) (Takes PartitionedDataFrame, DateT and today's date as input)
5.   Rebalance Portfolio (**P** period intervals) (Takes O)
6.   Function to change the value of most recent prices stored (Takes Numpy array as input)


You may want to know about Numpy's Corrcoef ([np.corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)) function!


In [ ]:
class PortFolio:
  def __init__(self, balance, numStocks, negCorr, prices):
    self.balance = balance
    self.numStocks = numStocks
    self.numStocks += GetBalanced(prices,GetMomentumBasedPriority(PartitionedData,DateToIndex,list(DateToIndex.keys())[T]),self.balance)
    self.prices = prices
    self.balance -= np.sum(self.numStocks * self.prices) * (1 + F)
    self.negCorr = negCorr

  def SellStock(self, index):
    self.balance += self.numStocks[index] * self.prices[index] * (1 - F)
    self.numStocks[index] = 0
  
  def BuyStock(self, index, number):
    self.balance -= number * self.prices[index] * (1 + F)

  def CalculateNetWorth(self):
    return self.balance + np.sum(self.numStocks * self.prices) * (1 - F)

  def ChangePricesTo(self, newPriceVector):
    self.prices = newPriceVector

  def ChangePairs(self):
    if self.negCorr == []:
      return
    for i in range(len(self.negCorr) - 1):
      x = Switch(self.negCorr[i][0],self.negCorr[i][1],today,PartitionedData)
      y = self.negCorr[i][0] + self.negCorr[i][1] - x
      credit = self.numStocks[y] * self.prices[y] * (1 - F)
      self.numStocks[y] = 0
      self.numStocks[x] += credit / (self.prices[x] * (1 + F))
  
  def RebalancePortfolio(self):
    TdaysAgo = datetime.date(int(today[0:4]),int(today[4:6]),int(today[6:])) + datetime.timedelta(days = -T)
    i = 0
    while i >= 0:
      x = str(TdaysAgo - datetime.timedelta(days = i)).replace('-','')
      if (x in DateToIndex.keys()):
        break
      i += 1
    temp = x
    Sum = [np.array(PartitionedData[DateToIndex[today] % 2926]['adjcp'])]
    for i in range((DateToIndex[temp] + 1), (DateToIndex[today]),30):
      Sum.append(np.array(PartitionedData[i % 2926]['adjcp']))
    corrCoef = np.corrcoef(np.array(Sum))
    copy = np.sort(corrCoef.flatten())
    for k in range(M):
      for i in range(corrCoef.shape[0]):
        for j in range(30):
          if abs(corrCoef[i][j] - copy[k]) < 1e-10:
            self.negCorr.append((i,j))
    self.negCorr = self.negCorr[0:M]
    copy = self.negCorr.copy()
    if self.negCorr == []:
      return
    for i in range(len(self.negCorr) - 1):
      for j in range(i + 1, len(self.negCorr)):
        x = list(self.negCorr[i])
        y = list(self.negCorr[j])
        if x[0] == y[0] or x[0] == y[1] or x[1] == y[0] or x[1] == y[1]:
          if self.negCorr[j] in copy and copy.count(self.negCorr[j]) > 1:
            copy.remove(self.negCorr[j])
    self.negCorr = copy.copy()
    self.ChangePairs()

With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [ ]:
Data = GetData("/path/to/DATA.csv")
List = PartitionData(Data)
PartitionedData = List[0]
DateToIndex = List[1]

myPortfolio = PortFolio(B * R,np.zeros(30, dtype = float),[],np.array(PartitionedData[int(list(DateToIndex.keys())[T]) % 2926]['adjcp']))
NetWorthAfterEachTrade = [myPortfolio.CalculateNetWorth() + B * (1 - R)]

for i in range((T + 1),len(PartitionedData)):
  today = list(DateToIndex.keys())[i]
  myPortfolio.ChangePricesTo(np.array(PartitionedData[int(today) % 2926]['adjcp']))
  NetWorthAfterEachTrade.append(myPortfolio.CalculateNetWorth() + B * (1 - R))
  if (i % T == 0):
    myPortfolio.RebalancePortfolio()
  if (i % P == 0):
    myPortfolio.ChangePairs()
  
VisualizeData(NetWorthAfterEachTrade[:D])

You may need to declare the PartitionedData list global, depending on your implementation of the Above snippet as well as the various functions in the Portfolio class.

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways (Profits, PercentageProfits etc.) and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

Feel free to copy-paste your code from the previous assignment!

In [ ]:
def VisualizeData(FinalData):
  plt.plot(FinalData)
  plt.show()

You may use this cell to write about your results/performance of the model!